# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import gmaps
from pprint import pprint
import json

# Import API key
from api_keys import geoapify_key
gmaps.configure(api_key = geoapify_key)

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")




In [75]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_df = pd.read_csv("output_data/cities.csv")


# Display sample data
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Paraul,42.7419,47.3553,52.34,84,100,9.08,RU,1682945486
1,1,Cape Town,-33.9258,18.4232,69.12,52,0,10.36,ZA,1682945402
2,2,Butaritari,3.0707,172.7902,82.65,80,15,15.93,KI,1682945487
3,3,Yellowknife,62.4560,-114.3525,43.32,56,100,12.66,CA,1682945487
4,4,Fairbanks,64.8378,-147.7164,35.60,69,40,3.44,US,1682945466


In [78]:

city_df["Humidity"] = city_df["Humidity"].astype(int)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [79]:
!jupyter nbextension enable --py gmaps



Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [81]:
import folium

# Define the map center and zoom level
center = (20.0, 0.0)
zoom_level = 2

# Create the map
m = folium.Map(location=center, zoom_start=zoom_level)

# Add markers to the map
for _, row in city_df.iterrows():
    popup = f"City: {row['City']}<br>Country: {row['Country']}<br>Humidity: {row['Humidity']}"
    folium.Marker([row['Lat'], row['Lng']], popup=popup, icon=folium.Icon(color='blue')).add_to(m)

# Save the map as an HTML file
m.save('my_map.html')


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [82]:
# Narrow down cities that fit criteria and drop any results with null values
good_weather_df = city_df.dropna()

# Display sample data
good_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Paraul,42.7419,47.3553,52.34,84,100,9.08,RU,1682945486
1,1,Cape Town,-33.9258,18.4232,69.12,52,0,10.36,ZA,1682945402
2,2,Butaritari,3.0707,172.7902,82.65,80,15,15.93,KI,1682945487
3,3,Yellowknife,62.4560,-114.3525,43.32,56,100,12.66,CA,1682945487
4,4,Fairbanks,64.8378,-147.7164,35.60,69,40,3.44,US,1682945466
...,...,...,...,...,...,...,...,...,...,...
570,617,Aykhal,66.0000,111.5000,28.29,96,100,28.61,RU,1682945625
571,618,Kenai,60.5544,-151.2583,39.29,72,100,6.91,US,1682945625
572,619,Nālūt,30.3333,10.8500,90.16,14,100,11.72,LY,1682945625
573,620,Abapó,-18.8333,-63.4667,67.69,87,100,1.74,BO,1682945626


In [83]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

good_weather_df = pd.DataFrame(good_weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (ideal_weather_df["Max Temp"] <= 80) & (ideal_weather_df["Max Temp"] > 70)
wind_speed = ideal_weather_df["Wind Speed"] < 10
cloudiness = ideal_weather_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
good_weather_df[max_temp & wind_speed & cloudiness]


,City,Max Temp,Wind Speed,Cloudiness
54,Arraial do Cabo,78.66,8.05,0
56,Esmeraldas,70.84,1.48,0
150,East London,72.46,8.72,0
296,Prata,76.78,0.94,0
303,Imbituba,72.86,1.63,0
352,Capinópolis,77.76,1.41,0
354,Santa Helena de Goiás,76.26,3.60,0
362,Manicoré,76.95,0.00,0
493,Qinzhou,75.43,7.52,0
542,Los Llanos de Aridane,76.75,8.05,0


### Step 3: Create a new DataFrame called `hotel_df`.

In [84]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_df [['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df["Hotel Name"] = ""

hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,Paraul,RU,42.7419,47.3553,84,
1,Cape Town,ZA,-33.9258,18.4232,52,
2,Butaritari,KI,3.0707,172.7902,80,
3,Yellowknife,CA,62.4560,-114.3525,56,
4,Fairbanks,US,64.8378,-147.7164,69,
...,...,...,...,...,...,...
570,Aykhal,RU,66.0000,111.5000,96,
571,Kenai,US,60.5544,-151.2583,72,
572,Nālūt,LY,30.3333,10.8500,14,
573,Abapó,BO,-18.8333,-63.4667,87,


In [63]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
#hotel_df['Hotel Name'] = pd.Series(dtype=str)

# display the updated DataFrame to verify the new column was added
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Paraul,RU,42.7419,47.3553,84,
1,Cape Town,ZA,-33.9258,18.4232,52,
2,Butaritari,KI,3.0707,172.7902,80,
3,Yellowknife,CA,62.4560,-114.3525,56,
4,Fairbanks,US,64.8378,-147.7164,69,
...,...,...,...,...,...,...
570,Aykhal,RU,66.0000,111.5000,96,
571,Kenai,US,60.5544,-151.2583,72,
572,Nālūt,LY,30.3333,10.8500,14,
573,Abapó,BO,-18.8333,-63.4667,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [85]:
# Define the API endpoint and parameters
url = "https://api.geoapify.com/v2/places"
params = {
    "categories": "accommodation.hotel",
    "limit": 10,
    "apiKey": geoapify_key
}

for i in range(len(hotel_df)):
    Lat = hotel_df.loc[i, "Lat"]
    Lon = hotel_df.loc[i, "Lng"]
    params["filter"]=f"circle:{Lon},{Lat},10000"
    params["bias"] = f"proximity:{Lon},{Lat}"
    response = requests.get(url, params=params).json()
    try:
        hotel_name = response["features"][0]["properties"]["name"]
        hotel_df.loc[i, "Hotel Name"] = hotel_name
    except:
        hotel_df.loc[i, "Hotel Name"] = "No Hotel Found"
    print(f"{hotel_df.loc[i,'Hotel Name']}")


No Hotel Found
Townhouse Hotel
No Hotel Found
No Hotel Found
Westmark Fairbanks Hotel
No Hotel Found
No Hotel Found
Humaita Quality Hotel
Grand Bel Air
Comfort Inn & Suites Redwood Country
Apart Hotel Aires del Beagle
Kap Farvel
Ocean Centre Hotel
Факел
No Hotel Found
Hostal La Mirada De Solitario George
Keleti International Resort
Page 63 hostel
No Hotel Found
Hotel Qaqortoq
Хатанга
Pono Kai Resort
Hotel Klaksvik
YHA Ahipara
No Hotel Found
Hotel Marlin
Woodbridge on the Derwent
Yamouri Inn
Quality Hotel Vøringfoss
Chez Bianca & Benoit
Hotel Lacolet
No Hotel Found
DeLuxe
Cristalina palace hotel
Kavieng Hotel
Aspen Suites Hotel
Paradise Inn
No Hotel Found
Nuevo Ostende
No Hotel Found
Shelikof Lodge
Hotel South Otago
No Hotel Found
Suita
Broadwater Beach Resort
No Hotel Found
Hotel Qaanaaq
Dolphin Bay Hotel
Protea Hotel
River Tides B&B, Norris Arm
La Portada
BEST WESTERN PLUS Pioneer Park Inn
Angmagssalik
Estalagem Ponta do Sol
No Hotel Found
Waterford Hotel
No Hotel Found
DoubleTree by 

Hotel Marajó
No Hotel Found
Hotel Hacienda Vieja Tomatlan
No Hotel Found
The Adventures Inn
No Hotel Found
No Hotel Found
Hotel Puerto Colombia
Hotel del Valle
Hotel São Miguel
山口楼
Parsi
Hotel Hero
No Hotel Found
Grand Hotel
Driftwood Inn & Homer Seaside Lodges
Castillo Real
Mega Inn
No Hotel Found
Parkside Inn
Cabañas
Hotel Ensueño
No Hotel Found
Senja hotell
No Hotel Found
No Hotel Found
No Hotel Found
Traveller's Rest
DoubleTree by Hilton Lubbock University Area
No Hotel Found
Los Caracoles
Fontana Suites
No Hotel Found
No Hotel Found
The Advenire, an Autograph Collection Hotel
Hotel Alexandre
Karratha International Hotel
No Hotel Found
SØMA
Aurora Alice Springs
Pousada Atlântico
Hostal Familiar Carmencita
Valle Aridane
Panorama Hotel
Black Rock Oceanfront Resort
Esplanade Hotel
Uyguluma Oteli
Blue Paloma Bed And Breakfast
The Cathedral Hotel
auberge la rose
Северяночка
Best Western Red River Inn
No Hotel Found
Well Garden
Hospedaje Calle del Sol
运城宾馆
Hostal Alameda
No Hotel Found
N

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [95]:
import plotly.graph_objs as go

# Initialize the figure with the mapbox style and center it on the first city in the city_df DataFrame
fig = go.Figure(go.Scattermapbox(
        lat=city_df["Lat"],
        lon=city_df["Lng"],
        mode='markers',
        marker=dict(size=10, color='blue'),
        text=city_data_df["City"],
        hovertemplate="<b>%{text}</b><br><br>" +
                      "Lat: %{lat:.2f}<br>" +
                      "Lng: %{lng:.2f}<br>" +
                      "<extra></extra>"
))

# Create a list to hold the hover text
hover_text = []

# Iterate through the hotel_df DataFrame and add the hotel name and country to the hover text
for index, row in hotel_df.iterrows():
    hover_text.append(f"{row['Hotel Name']}, {row['Country']}")

# Create a Scattermapbox trace for the hotels
hotel_trace = go.Scattermapbox(
    lat=hotel_df["Lat"],
    lon=hotel_df["Lng"],
    mode="markers",
    marker=dict(
        size=10,
        color="blue",
    ),
    text=hover_text, # Use the hover_text list for the hover text
)

# Add the trace to the figure and show the figure
fig.add_trace(hotel_trace)
fig.show()


In [89]:
import plotly.graph_objs as go

# Create a list to hold the hover text
hover_text = []

# Iterate through the hotel_df DataFrame and add the hotel name and country to the hover text
for index, row in hotel_df.iterrows():
    hover_text.append(f"{row['Hotel Name']}, {row['Country']}")

# Create a Scattermapbox trace for the hotels
hotel_trace = go.Scattermapbox(
    lat=hotel_df["Lat"],
    lon=hotel_df["Lng"],
    marker=dict(
        size=10,
        color="red",
    ),
    text=hover_text, # Use the hover_text list for the hover text
)

# Add the trace to the figure and show the figure
fig.add_trace(hotel_trace)
fig.show()
